## Drifters and Cmems data Notebook

### Introduction
This notebook aims to interpolate the wind field from CMEMS onto the drifter positions and calculate the surface Ekman current, and then add it to the geostrophic current.

### Data Sources
- Drifter Data:
  - Source: NOAA
  - Collection Frequency: Every 6 hours
  - Total Number of Drifters: 221

- CMEMS Data:
  - Source: Copernicus Marine
  - Collection Frequency: Every 12 hours
  - Data Product: SEALEVEL_GLO_PHY_L4_MY_008_047

### Tasks

#### 1. Interpolation and Ekman current

- Interpolating the wind field from CMEMS onto the drifter positions and calculating the surface Ekman current.

#### 2. Adding the surface Ekman current to the geostrophic current.

#### 1. Interpolation and Ekman current

## Import Library

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy.interpolate import interp2d

import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature
from cartopy import config
import matplotlib.animation

## Read Drifter Data

In [2]:
fn='./drifter_6hour_qc_893c_d15d_c4b7_U1711967648742.nc';
ds=xr.open_dataset(fn);

## Extract Coordinates and Variables

In [3]:
ve_curr = ds.ve.values
vn_curr = ds.vn.values
sst = ds.sst.values
lon = ds.longitude.values
lat = ds.latitude.values
time = ds.time.values

## Creation of Pandas Array for Drifter Data

In [4]:
data = {
    'longitude': lon,
    'latitude' : lat,
    'time'     : time,
    've'       : ve_curr,
    'vn'       : vn_curr,
    'sst'      : sst
}

df = pd.DataFrame(data)
df

,longitude,latitude,time,ve,vn,sst
0,-14.478,-18.548,2023-01-01 00:00:00,-0.13760,-0.07635,23.612
1,-14.499,-18.556,2023-01-01 06:00:00,-0.09969,0.01322,23.564
2,-14.519,-18.543,2023-01-01 12:00:00,-0.15719,0.08193,23.699
3,-14.563,-18.524,2023-01-01 18:00:00,-0.26155,0.03533,23.856
4,-14.626,-18.529,2023-01-02 00:00:00,-0.23922,-0.07837,23.717
...,...,...,...,...,...,...
78378,-22.006,-19.946,2023-06-11 12:00:00,-0.03853,-0.05076,25.189
78379,-22.013,-19.961,2023-06-11 18:00:00,-0.05150,-0.06262,25.179
78380,-22.027,-19.971,2023-06-12 00:00:00,-0.06030,-0.04773,25.133
78381,-22.038,-19.980,2023-06-12 06:00:00,-0.06036,-0.05688,25.093


## Selecting Drifter Data

In [8]:
# Test: Our selected drifter has 258 observations. However, we will limit it to 158 observations because the coordinates 
# of our CMEMS data are 158 in dimension, and the linear interpolation method we use requires it in this situation.

df_1 = df[19675:19833]
df_1

,longitude,latitude,time,ve,vn,sst
19675,-63.153,18.267,2023-01-01 00:00:00,0.00100,-0.00273,26.233
19676,-63.152,18.266,2023-01-01 06:00:00,0.00033,0.00000,26.419
19677,-63.152,18.267,2023-01-01 12:00:00,-0.00107,0.00254,26.449
19678,-63.153,18.267,2023-01-01 18:00:00,-0.00033,0.00001,26.324
19679,-63.153,18.267,2023-01-02 00:00:00,0.00074,-0.00087,26.276
...,...,...,...,...,...,...
19828,-63.148,18.252,2023-02-08 06:00:00,-0.00308,-0.00033,25.941
19829,-63.152,18.267,2023-02-08 12:00:00,0.01344,0.05190,25.946
19830,-63.142,18.272,2023-02-08 18:00:00,-0.01471,0.02242,25.783
19831,-63.158,18.276,2023-02-09 00:00:00,-0.02561,-0.01237,25.781


## Now Let's Interpolating the wind field from CMEMS onto the drifter positions

In [9]:
# Load the DataFrame containing information about the cordinate
df_inertial_wave = df[19675:19833] 

# Load data from the NetCDF file containing the U and V wind components
ds_wind = xr.open_dataset("cmems_obs-wind_glo_phy_my_l4_P1M_1715561830429.nc")  # Make sure to provide the correct file path

# Select the U and V wind variables
u_wind = ds_wind['eastward_wind']
v_wind = ds_wind['northward_wind']

# Interpolate wind data to the same spatial positions as the inertial wave data
u_interp = u_wind.interp(latitude=df_inertial_wave['latitude'], longitude=df_inertial_wave['longitude'], time=df_inertial_wave['time'], method='linear')
v_interp = v_wind.interp(latitude=df_inertial_wave['latitude'], longitude=df_inertial_wave['longitude'], time=df_inertial_wave['time'], method='linear')


## Calculation of surface Ekman current

## Calculation of Ekman current speed:

The speed of Ekman current can be calculated using the Ekman equation:

$$
V_e = \frac{\rho f}{\tau}
$$

Where:
- $ V_e $ is the speed of the Ekman current.
- $ \tau $ is the wind stress, which is the force exerted by the wind on the water surface.
- $ \rho $ is the water density.
- $ f $ is the Coriolis parameter, which depends on latitude.

## Zonal case

In [17]:
# Interpolated wind data
# Suppose u_wind contains interpolated zonal wind data

# Constants
rho = 1025  # Water density in kg/m^3
f = 2 * np.pi / (24 * 3600) * np.sin(np.deg2rad(u_interp.latitude))  # Coriolis parameter

# Calculation of wind stress (hypothetical)
tau = 0.1  # Example wind stress in N/m^2

# Calculation of Ekman current speed
ue = tau / (rho * f)

# Ekman current direction
# For example, Ekman current direction is at 90 degrees to the wind direction
# For specific direction, it depends on the hemisphere (to the right in the northern hemisphere, to the left in the southern hemisphere)

# Example of displaying results
# print("Zonal Ekman current speed (m/s):", ue)

# Meridional case

In [18]:
# Interpolated wind data
# Suppose v_wind contains interpolated meridional wind data

# Constants
rho = 1025  # Water density in kg/m^3
f = 2 * np.pi / (24 * 3600) * np.sin(np.deg2rad(v_interp.latitude))  # Coriolis parameter

# Calculation of wind stress (hypothetical)
tau = 0.1  # Example wind stress in N/m^2

# Calculation of Ekman current speed
ve = tau / (rho * f)

# Ekman current direction
# For example, Ekman current direction is at 90 degrees to the wind direction
# For specific direction, it depends on the hemisphere (to the right in the northern hemisphere, to the left in the southern hemisphere)

# Example of displaying results
# print("Zonal Ekman current speed (m/s):", ve)

#### 2. Adding the surface Ekman current to the geostrophic current.

## ugeos plus Ekman zonal current

In [26]:
U_geos_ekman= df_1['ve'] + ue

## vgeos plus Ekman meridional current

In [25]:
V_geos_ekman= df_1['vn'] + ve